# Mars image recognition

#### Don't use gpu

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

#### Includes

In [2]:
import pandas as pd
import numpy as np
import PIL
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, models

#### Load data from files

In [3]:
data_folder = '../data/hirise-map-proj-v3_2/'

In [4]:
def extract_metadata() -> pd.DataFrame:
    images_and_labels = None
    with open(data_folder + 'labels-map-proj_v3_2_train_val_test.txt',
              'r') as f:
        images_and_labels = f.readlines()
    images_and_labels = pd.DataFrame(images_and_labels).replace(
        '\n', '', regex=True)[0].str.split(' ', expand=True)
    images_and_labels.rename(columns={
        0: 'file_name',
        1: 'label',
        2: 'type'
    },
                             inplace=True)
    return images_and_labels

In [5]:
metadata = extract_metadata()

In [6]:
train_data = metadata[metadata['type'] == 'train']
test_data = metadata[metadata['type'] == 'test']
val_data = metadata[metadata['type'] == 'val']

In [7]:
def image_to_np(image: pd.Series) -> np.array:
    return np.asarray(
        PIL.Image.open(data_folder + 'map-proj-v3_2/' + image['file_name']))

In [8]:
def get_data(data: pd.DataFrame) -> tuple:
    labels = data['label']
    data = data.apply(image_to_np, axis=1)
    return np.stack(data.to_numpy()), np.stack(labels.to_numpy()).astype(float)

In [9]:
train_data, train_labels = get_data(train_data)
test_data, test_labels = get_data(test_data)
val_data, val_labels = get_data(val_data)

#### Train the model

In [10]:
model = models.Sequential()
model.add(
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(227, 227, 1)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Dropout(.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.AvgPool2D((2, 2)))
model.add(layers.Dropout(.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

2022-03-23 23:51:13.034275: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-23 23:51:13.034331: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ArchPC
2022-03-23 23:51:13.034341: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ArchPC
2022-03-23 23:51:13.034444: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.54.0
2022-03-23 23:51:13.034479: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.54.0
2022-03-23 23:51:13.034488: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.54.0
2022-03-23 23:51:13.035039: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

In [11]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(8))

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
history = model.fit(train_data, train_labels, epochs=10, batch_size=52)

2022-03-23 23:51:13.169206: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2630967682 exceeds 10% of free system memory.


Epoch 1/10
 67/982 [=>............................] - ETA: 1:13:11 - loss: 1.2569 - accuracy: 0.7712

#### Check model accuracy and performance

In [ ]:
val_loss, val_acc = model.evaluate(val_data, val_labels, verbose=2)
print(val_loss)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)
print(test_acc)

#### Save model 

In [ ]:
model.save('./model')

#### Load model

In [ ]:
model = models.load_model('./model')